# 🚀 네이버 뉴스 기반 한국어 RAG 튜토리얼

## 📚 학습 목표
이 튜토리얼을 통해 다음을 배우게 됩니다:

1. **RAG(Retrieval-Augmented Generation)의 기본 개념**
2. **LangChain을 활용한 RAG 시스템 구축**
3. **한국어 텍스트 처리 및 임베딩**
4. **네이버 뉴스를 활용한 실시간 질답 시스템 개발**

## 🔍 RAG란 무엇인가요?

**RAG(Retrieval-Augmented Generation)**는 외부 지식 베이스에서 관련 정보를 검색(Retrieval)하여 언어모델의 생성(Generation) 능력을 보강하는 기술입니다.

### 🤔 왜 RAG가 필요한가요?

1. **최신 정보 부족**: LLM은 훈련 시점 이후의 정보를 모릅니다
2. **도메인 특화 지식**: 특정 분야의 전문 지식이 부족할 수 있습니다
3. **환각(Hallucination)**: 잘못된 정보를 그럴듯하게 생성할 수 있습니다

### ✅ RAG의 장점

- 📈 **최신 정보 활용**: 실시간 데이터 반영 가능
- 🎯 **정확도 향상**: 검증된 소스에서 정보 추출
- 💰 **비용 효율성**: 전체 모델 재훈련 불필요
- 🔄 **유연성**: 지식 베이스 쉽게 업데이트 가능


## 🏗️ RAG 시스템 아키텍처

RAG 시스템은 크게 **2개의 주요 단계**로 구성됩니다:

### 📋 1단계: 사전 작업(Pre-processing) - 지식 베이스 구축

```
[문서] → [분할] → [임베딩] → [벡터DB 저장]
```

1. **문서 로드**: 원본 문서나 데이터를 시스템에 로드
2. **텍스트 분할**: 큰 문서를 작은 청크(Chunk)로 나누기
3. **임베딩**: 텍스트를 벡터 형태로 변환
4. **저장**: 벡터 데이터베이스에 저장

### 🔄 2단계: 실행 시간(Runtime) - 질의응답 수행

```
[질문] → [검색] → [프롬프트 생성] → [LLM] → [답변]
```

1. **질문 입력**: 사용자가 질문을 입력
2. **관련 문서 검색**: 벡터 유사도를 기반으로 관련 문서 검색
3. **프롬프트 생성**: 검색된 문서와 질문을 조합하여 프롬프트 생성
4. **LLM 생성**: 언어모델이 최종 답변 생성

---

## 🛠️ 실습 환경 준비


### 📦 필요한 라이브러리 설치

먼저 필요한 라이브러리들을 설치해보겠습니다. 각 라이브러리의 역할을 알아봅시다:

- **langchain**: RAG 파이프라인 구축을 위한 핵심 프레임워크
- **langchain-openai**: OpenAI API 연동
- **langchain-community**: 커뮤니티에서 제공하는 추가 기능들
- **beautifulsoup4**: 웹 페이지 파싱 (네이버 뉴스 크롤링용)
- **faiss-cpu**: 고속 벡터 유사도 검색 엔진
- **python-dotenv**: 환경변수 관리

실제 환경에서는 아래 명령어로 설치하세요:
```bash
pip install langchain langchain-openai langchain-community langchain-text-splitters
pip install beautifulsoup4 faiss-cpu python-dotenv
```


In [1]:
# 🔧 환경 설정 및 라이브러리 import
import os
from dotenv import load_dotenv
import bs4
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

print("✅ 모든 라이브러리가 성공적으로 import되었습니다!")


USER_AGENT environment variable not set, consider setting it to identify your requests.


✅ 모든 라이브러리가 성공적으로 import되었습니다!


### 🔑 API 키 설정

OpenAI API를 사용하기 위해 API 키를 설정해야 합니다.

**⚠️ 중요**: 
- OpenAI API 키는 [https://platform.openai.com/api-keys](https://platform.openai.com/api-keys)에서 발급받을 수 있습니다.
- `.env` 파일에 `OPENAI_API_KEY=your_api_key_here` 형태로 저장하세요.
- **절대 코드에 직접 API 키를 넣지 마세요!** (보안상 위험합니다)


In [2]:
# API 키 로드
load_dotenv()

# API 키가 제대로 설정되었는지 확인
if os.getenv("OPENAI_API_KEY"):
    print("✅ OpenAI API 키가 정상적으로 로드되었습니다!")
else:
    print("❌ OpenAI API 키가 설정되지 않았습니다.")
    print("💡 .env 파일에 OPENAI_API_KEY=your_api_key 를 추가하세요.")


✅ OpenAI API 키가 정상적으로 로드되었습니다!


---

## 🏁 RAG 파이프라인 구축 시작!

이제 본격적으로 RAG 시스템을 단계별로 구축해보겠습니다.

## 📰 1단계: 문서 로드 (Document Loading)

첫 번째 단계는 네이버 뉴스 기사를 로드하는 것입니다.

### 🔍 WebBaseLoader 이해하기

`WebBaseLoader`는 웹 페이지의 내용을 추출하는 도구입니다. 네이버 뉴스 페이지에서 실제 기사 내용만 추출하기 위해 CSS 선택자를 사용합니다.

**주요 매개변수:**
- `web_paths`: 크롤링할 웹페이지 URL
- `bs_kwargs`: BeautifulSoup 설정 (파싱할 HTML 요소 지정)


In [3]:
# 네이버 뉴스 URL (예시 - 실제 사용시 최신 뉴스로 변경하세요)
news_url = "https://n.news.naver.com/article/437/0000378416"

print(f"📰 로딩할 뉴스 URL: {news_url}")
print("🔄 뉴스 기사를 로딩 중입니다...")

# WebBaseLoader 설정
# 네이버 뉴스의 제목과 본문만 추출하기 위한 CSS 선택자 설정
loader = WebBaseLoader(
    web_paths=(news_url,),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)

# 문서 로드 실행
docs = loader.load()

print(f"✅ 문서 로드 완료! 총 {len(docs)}개 문서가 로드되었습니다.")

# 로드된 문서의 일부 내용 확인
if docs:
    print("\n📄 로드된 문서 미리보기:")
    print("-" * 50)
    print(docs[0].page_content[:300] + "...")
    print("-" * 50)
    print(f"📊 전체 문서 길이: {len(docs[0].page_content)} 문자")


📰 로딩할 뉴스 URL: https://n.news.naver.com/article/437/0000378416
🔄 뉴스 기사를 로딩 중입니다...
✅ 문서 로드 완료! 총 1개 문서가 로드되었습니다.

📄 로드된 문서 미리보기:
--------------------------------------------------

출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책


[앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나섰습니다. 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다. 선거 때면 나타나서 아이 낳으면 현금 주겠다고 밝힌 사람이 있었죠. 과거에는...
--------------------------------------------------
📊 전체 문서 길이: 1194 문자


## ✂️ 2단계: 텍스트 분할 (Text Splitting)

### 🤔 왜 텍스트를 분할해야 할까요?

1. **토큰 제한**: LLM은 한 번에 처리할 수 있는 토큰 수가 제한되어 있습니다
2. **검색 정확도**: 작은 청크일수록 더 정확한 검색이 가능합니다
3. **의미 단위**: 관련된 내용끼리 묶어서 처리하면 더 좋은 결과를 얻을 수 있습니다

### 🔧 RecursiveCharacterTextSplitter 매개변수

- **chunk_size**: 각 청크의 최대 문자 수
- **chunk_overlap**: 인접한 청크 간 겹치는 문자 수 (문맥 연결을 위해)

**💡 팁**: 
- chunk_size가 클수록: 더 많은 문맥 정보, 하지만 검색 정확도 감소
- chunk_size가 작을수록: 정확한 검색, 하지만 문맥 정보 부족


In [4]:
# 텍스트 분할기 설정
chunk_size = 1000  # 각 청크의 최대 크기
chunk_overlap = 100  # 청크 간 겹치는 부분

print(f"🔧 텍스트 분할 설정:")
print(f"   - 청크 크기: {chunk_size} 문자")
print(f"   - 겹침: {chunk_overlap} 문자")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

# 문서 분할 실행
print("\n✂️ 문서를 청크로 분할하는 중...")
splits = text_splitter.split_documents(docs)

print(f"✅ 분할 완료! 총 {len(splits)}개의 청크가 생성되었습니다.")

# 분할된 청크들 살펴보기
print("\n📝 생성된 청크들 미리보기:")
for i, chunk in enumerate(splits[:3]):  # 처음 3개 청크만 보기
    print(f"\n--- 청크 {i+1} ---")
    print(f"길이: {len(chunk.page_content)} 문자")
    print(f"내용: {chunk.page_content[:150]}...")

print(f"\n📊 청크 크기 분포:")
chunk_lengths = [len(chunk.page_content) for chunk in splits]
print(f"   - 최소: {min(chunk_lengths)} 문자")
print(f"   - 최대: {max(chunk_lengths)} 문자")
print(f"   - 평균: {sum(chunk_lengths)/len(chunk_lengths):.0f} 문자")


🔧 텍스트 분할 설정:
   - 청크 크기: 1000 문자
   - 겹침: 100 문자

✂️ 문서를 청크로 분할하는 중...
✅ 분할 완료! 총 3개의 청크가 생성되었습니다.

📝 생성된 청크들 미리보기:

--- 청크 1 ---
길이: 31 문자
내용: 출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책...

--- 청크 2 ---
길이: 996 문자
내용: [앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나...

--- 청크 3 ---
길이: 249 문자
내용: 출산장려책은 점점 확산하는 분위기입니다.법정기간보다 육아휴직을 길게 주거나, 남성 직원의 육아휴직을 의무화한 곳도 있습니다.사내 어린이집을 밤 10시까지 운영하고 셋째를 낳으면 무조건 승진시켜 주기도 합니다.한 회사는 지난해 네쌍둥이를 낳은 직원에 의료비를 지원해 관심...

📊 청크 크기 분포:
   - 최소: 31 문자
   - 최대: 996 문자
   - 평균: 425 문자


## 🔢 3단계: 임베딩 생성 (Embedding)

### 🧠 임베딩이란?

임베딩은 텍스트를 **숫자 벡터로 변환**하는 과정입니다. 컴퓨터는 숫자만 이해할 수 있기 때문에, 텍스트의 의미를 벡터 공간에서 표현합니다.

**예시:**
- "사과" → [0.1, 0.8, 0.3, ...]
- "과일" → [0.2, 0.7, 0.4, ...]
- "자동차" → [0.9, 0.1, 0.2, ...]

유사한 의미를 가진 단어들은 벡터 공간에서 **가까운 위치**에 놓입니다.

### 🔍 OpenAI 임베딩의 특징

- **높은 품질**: 다양한 언어와 도메인에서 우수한 성능
- **다차원**: 보통 1536차원의 벡터로 표현
- **한국어 지원**: 한국어 텍스트도 잘 처리합니다


In [5]:
# OpenAI 임베딩 모델 초기화
print("🔢 임베딩 모델을 초기화하는 중...")
embeddings = OpenAIEmbeddings()

print("✅ OpenAI 임베딩 모델이 준비되었습니다!")
print("📝 사용 모델: text-embedding-ada-002")

# 임베딩 테스트 (간단한 예시)
test_text = "안녕하세요, 한국어 임베딩 테스트입니다."
print(f"\n🧪 임베딩 테스트 문장: '{test_text}'")

# 실제로 임베딩 해보기 (시간이 조금 걸릴 수 있습니다)
try:
    test_embedding = embeddings.embed_query(test_text)
    print(f"✅ 임베딩 생성 성공!")
    print(f"📊 벡터 차원: {len(test_embedding)}")
    print(f"🔢 벡터 일부 (처음 5개): {test_embedding[:5]}")
except Exception as e:
    print(f"❌ 임베딩 생성 실패: {e}")
    print("💡 API 키를 확인해주세요.")


🔢 임베딩 모델을 초기화하는 중...
✅ OpenAI 임베딩 모델이 준비되었습니다!
📝 사용 모델: text-embedding-ada-002

🧪 임베딩 테스트 문장: '안녕하세요, 한국어 임베딩 테스트입니다.'
✅ 임베딩 생성 성공!
📊 벡터 차원: 1536
🔢 벡터 일부 (처음 5개): [-0.0014216081472113729, -0.015253019519150257, -0.0035998146049678326, -0.03269778564572334, -0.008422928862273693]


## 💾 4단계: 벡터 저장소 생성 (Vector Store)

### 🗃️ 벡터 저장소란?

벡터 저장소는 임베딩된 텍스트 벡터들을 **효율적으로 저장하고 검색**할 수 있는 데이터베이스입니다.

### 🚀 FAISS의 장점

**FAISS(Facebook AI Similarity Search)**는 Meta에서 개발한 고성능 벡터 검색 라이브러리입니다:

- ⚡ **빠른 검색**: 수백만 개의 벡터에서도 밀리초 단위 검색
- 🎯 **높은 정확도**: 코사인 유사도 기반 정확한 검색
- 💻 **메모리 효율성**: 대용량 데이터도 효율적으로 처리
- 🔄 **쉬운 사용**: LangChain과 완벽 호환

### 🔍 검색 원리

1. 사용자 질문을 임베딩으로 변환
2. 저장된 벡터들과 유사도 계산  
3. 가장 유사한 상위 K개 문서 반환


In [6]:
# 벡터 저장소 생성 (시간이 조금 걸릴 수 있습니다)
print("💾 벡터 저장소 생성 중...")
print("   - 모든 텍스트 청크를 임베딩으로 변환하고 있습니다...")
print("   - 이 과정은 텍스트 양에 따라 1-2분 정도 소요될 수 있습니다...")

try:
    # FAISS 벡터스토어 생성
    vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
    
    print("✅ 벡터 저장소 생성 완료!")
    print(f"📊 저장된 문서 수: {len(splits)}개")
    
    # 간단한 검색 테스트
    print("\n🔍 벡터 저장소 검색 테스트:")
    test_query = "정부 정책"
    search_results = vectorstore.similarity_search(test_query, k=2)
    
    print(f"검색어: '{test_query}'")
    print(f"검색 결과: {len(search_results)}개 문서 발견")
    
    for i, result in enumerate(search_results):
        print(f"\n--- 검색 결과 {i+1} ---")
        print(f"내용: {result.page_content[:100]}...")
        
except Exception as e:
    print(f"❌ 벡터 저장소 생성 실패: {e}")
    print("💡 API 키와 인터넷 연결을 확인해주세요.")


💾 벡터 저장소 생성 중...
   - 모든 텍스트 청크를 임베딩으로 변환하고 있습니다...
   - 이 과정은 텍스트 양에 따라 1-2분 정도 소요될 수 있습니다...
✅ 벡터 저장소 생성 완료!
📊 저장된 문서 수: 3개

🔍 벡터 저장소 검색 테스트:
검색어: '정부 정책'
검색 결과: 2개 문서 발견

--- 검색 결과 1 ---
내용: 출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책...

--- 검색 결과 2 ---
내용: 출산장려책은 점점 확산하는 분위기입니다.법정기간보다 육아휴직을 길게 주거나, 남성 직원의 육아휴직을 의무화한 곳도 있습니다.사내 어린이집을 밤 10시까지 운영하고 셋째를 낳으면 무...


## 🔍 5단계: 검색기 생성 (Retriever)

### 🎯 검색기의 역할

검색기는 사용자의 질문과 가장 관련성 높은 문서 청크들을 찾아주는 중요한 역할을 합니다.

### ⚙️ 검색기 설정 옵션

- **k**: 반환할 문서의 개수 (기본값: 4)
- **search_type**: 검색 유형
  - `"similarity"`: 유사도 기반 검색 (기본값)
  - `"mmr"`: 최대 한계 적합성 (다양성 고려)
- **score_threshold**: 유사도 임계값 (이 값 이상의 문서만 반환)

### 💡 검색 성능 튜닝 팁

- **k값이 클수록**: 더 많은 정보, 하지만 노이즈 증가 가능
- **k값이 작을수록**: 정확한 정보, 하지만 정보 부족 가능
- **일반적으로 3-5개가 적당합니다**


In [7]:
# 검색기 생성
print("🔍 검색기 생성 중...")

# 기본 설정으로 검색기 생성
retriever = vectorstore.as_retriever()

print("✅ 기본 검색기 생성 완료!")
print("📋 기본 설정:")
print("   - 검색 방식: 유사도 기반")
print("   - 반환 문서 수: 4개")

# 커스텀 설정으로 검색기 생성 (선택사항)
print("\n🔧 커스텀 검색기도 만들어보겠습니다:")
custom_retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # 3개 문서만 반환
)

print("✅ 커스텀 검색기 생성 완료!")
print("📋 커스텀 설정:")
print("   - 검색 방식: 유사도 기반")
print("   - 반환 문서 수: 3개")

# 검색기 테스트
print("\n🧪 검색기 성능 테스트:")
test_questions = [
    "이 기사의 주요 내용은 무엇인가요?",
    "정부의 정책은 무엇인가요?",
    "주요 인물은 누구인가요?"
]

for question in test_questions:
    print(f"\n❓ 질문: {question}")
    
    # 기본 검색기로 검색
    results = retriever.invoke(question)
    print(f"🔍 기본 검색기 결과: {len(results)}개 문서")
    
    # 첫 번째 결과만 미리보기
    if results:
        print(f"📄 첫 번째 문서: {results[0].page_content[:80]}...")

print("\n✅ 검색기 테스트 완료!")


🔍 검색기 생성 중...
✅ 기본 검색기 생성 완료!
📋 기본 설정:
   - 검색 방식: 유사도 기반
   - 반환 문서 수: 4개

🔧 커스텀 검색기도 만들어보겠습니다:
✅ 커스텀 검색기 생성 완료!
📋 커스텀 설정:
   - 검색 방식: 유사도 기반
   - 반환 문서 수: 3개

🧪 검색기 성능 테스트:

❓ 질문: 이 기사의 주요 내용은 무엇인가요?
🔍 기본 검색기 결과: 3개 문서
📄 첫 번째 문서: 출산장려책은 점점 확산하는 분위기입니다.법정기간보다 육아휴직을 길게 주거나, 남성 직원의 육아휴직을 의무화한 곳도 있습니다.사내 어린이집을 밤 ...

❓ 질문: 정부의 정책은 무엇인가요?
🔍 기본 검색기 결과: 3개 문서
📄 첫 번째 문서: 출산장려책은 점점 확산하는 분위기입니다.법정기간보다 육아휴직을 길게 주거나, 남성 직원의 육아휴직을 의무화한 곳도 있습니다.사내 어린이집을 밤 ...

❓ 질문: 주요 인물은 누구인가요?
🔍 기본 검색기 결과: 3개 문서
📄 첫 번째 문서: 출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책...

✅ 검색기 테스트 완료!


## 📝 6단계: 프롬프트 생성 (Prompt Engineering)

### 🎭 프롬프트의 중요성

프롬프트는 AI에게 **"어떻게 답변할지"**를 알려주는 지침서입니다. 좋은 프롬프트는 다음을 포함해야 합니다:

1. **명확한 역할 정의**: AI가 어떤 역할을 해야 하는지
2. **컨텍스트 활용 지침**: 검색된 문서를 어떻게 사용할지  
3. **답변 형식 가이드**: 어떤 형태로 답변할지
4. **제약사항**: 모르는 경우 어떻게 답할지

### 🇰🇷 한국어 프롬프트 설계 포인트

- **정중한 톤**: 한국어의 존댓말 문화 반영
- **기술 용어 보존**: 번역하지 말고 원어 그대로 사용
- **명확한 안내**: 정보가 없을 때의 대응 방식 명시


In [8]:
# 한국어 RAG를 위한 프롬프트 템플릿 생성
print("📝 한국어 RAG 프롬프트 생성 중...")

prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 
당신의 임무는 주어진 문맥(context)에서 주어진 질문(question)에 답하는 것입니다.

검색된 다음 문맥(context)을 사용하여 질문(question)에 답하세요. 
만약, 주어진 문맥(context)에서 답을 찾을 수 없다면, 
`주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다`라고 답하세요.

한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

print("✅ 프롬프트 템플릿 생성 완료!")

# 프롬프트 구조 분석
print("\n📋 프롬프트 구조 분석:")
print("1. 역할 정의: '친절한 AI 어시스턴트'")
print("2. 임무 설명: '문맥에서 질문에 답하기'")
print("3. 사용 지침: '검색된 문맥 활용'")
print("4. 제약사항: '모르면 솔직히 말하기'")
print("5. 언어 설정: '한글 답변, 기술용어 보존'")
print("6. 입력 변수: question(질문), context(문맥)")

# 프롬프트 템플릿 미리보기
sample_question = "이 뉴스의 주요 내용은 무엇인가요?"
sample_context = "샘플 문맥입니다..."

print(f"\n🔍 프롬프트 템플릿 미리보기:")
print("-" * 50)
formatted_prompt = prompt.format(question=sample_question, context=sample_context)
print(formatted_prompt)
print("-" * 50)


📝 한국어 RAG 프롬프트 생성 중...
✅ 프롬프트 템플릿 생성 완료!

📋 프롬프트 구조 분석:
1. 역할 정의: '친절한 AI 어시스턴트'
2. 임무 설명: '문맥에서 질문에 답하기'
3. 사용 지침: '검색된 문맥 활용'
4. 제약사항: '모르면 솔직히 말하기'
5. 언어 설정: '한글 답변, 기술용어 보존'
6. 입력 변수: question(질문), context(문맥)

🔍 프롬프트 템플릿 미리보기:
--------------------------------------------------
당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 
당신의 임무는 주어진 문맥(context)에서 주어진 질문(question)에 답하는 것입니다.

검색된 다음 문맥(context)을 사용하여 질문(question)에 답하세요. 
만약, 주어진 문맥(context)에서 답을 찾을 수 없다면, 
`주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다`라고 답하세요.

한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question: 
이 뉴스의 주요 내용은 무엇인가요? 

#Context: 
샘플 문맥입니다... 

#Answer:
--------------------------------------------------


## 🤖 7단계: 언어모델 설정 (LLM)

### 🧠 ChatOpenAI 모델 선택

여러 OpenAI 모델 중에서 우리의 용도에 맞는 모델을 선택해보겠습니다:

| 모델 | 특징 | 속도 | 비용 | 추천 용도 |
|------|------|------|------|-----------|
| **gpt-4o-mini** | 높은 성능, 저렴한 비용 | 빠름 | 저렴 | ✅ **RAG에 최적** |
| gpt-4o | 최고 성능 | 보통 | 비싼편 | 복잡한 추론 |
| gpt-3.5-turbo | 기본 성능 | 매우 빠름 | 매우 저렴 | 간단한 대화 |

### ⚙️ 주요 매개변수

- **model_name**: 사용할 모델 선택
- **temperature**: 창의성 조절 (0: 일관적, 1: 창의적)
- **max_tokens**: 최대 응답 길이 제한


In [9]:
# 언어모델 설정
print("🤖 ChatOpenAI 언어모델 설정 중...")

# RAG에 최적화된 설정
llm = ChatOpenAI(
    model_name="gpt-4o-mini",  # 성능과 비용의 균형이 좋은 모델
    temperature=0              # 일관된 답변을 위해 창의성을 낮게 설정
)

print("✅ 언어모델 설정 완료!")
print("📋 설정된 모델 정보:")
print("   - 모델: gpt-4o-mini")
print("   - Temperature: 0 (일관된 답변)")
print("   - 용도: 질문-답변 시스템에 최적화")

# 모델 간단 테스트
print("\n🧪 언어모델 테스트:")
test_prompt = "안녕하세요! 한국어로 간단하게 자기소개를 해주세요."

try:
    response = llm.invoke(test_prompt)
    print(f"💬 모델 응답: {response.content}")
    print("✅ 언어모델이 정상적으로 작동합니다!")
except Exception as e:
    print(f"❌ 언어모델 테스트 실패: {e}")
    print("💡 API 키와 인터넷 연결을 확인해주세요.")


🤖 ChatOpenAI 언어모델 설정 중...
✅ 언어모델 설정 완료!
📋 설정된 모델 정보:
   - 모델: gpt-4o-mini
   - Temperature: 0 (일관된 답변)
   - 용도: 질문-답변 시스템에 최적화

🧪 언어모델 테스트:
💬 모델 응답: 안녕하세요! 저는 AI 언어 모델입니다. 다양한 주제에 대해 질문에 답하고 정보를 제공하는 역할을 하고 있습니다. 여러분의 궁금증을 해결하는 데 도움을 드릴 수 있어 기쁩니다. 무엇을 도와드릴까요?
✅ 언어모델이 정상적으로 작동합니다!


## 🔗 8단계: RAG 체인 생성 (Chain Assembly)

### 🎯 체인이란?

**체인(Chain)**은 여러 컴포넌트를 연결하여 하나의 파이프라인을 만드는 LangChain의 핵심 개념입니다.

### 🔄 우리 RAG 체인의 구조

```
질문 입력 → 검색기 → 프롬프트 → LLM → 출력 파서 → 최종 답변
```

각 단계의 역할:
1. **RunnablePassthrough**: 질문을 그대로 전달
2. **retriever**: 관련 문서 검색
3. **prompt**: 검색된 문서와 질문을 조합
4. **llm**: AI 모델이 답변 생성
5. **StrOutputParser**: 출력을 문자열로 변환

### 🧩 LCEL (LangChain Expression Language)

`|` 연산자를 사용하여 컴포넌트들을 파이프라인처럼 연결합니다:

```python
chain = input | component1 | component2 | output
```


In [10]:
# RAG 체인 생성 - 모든 컴포넌트를 연결합니다!
print("🔗 RAG 체인 생성 중...")

# 체인 구성: 각 단계를 파이프라인으로 연결
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}  # 입력: 검색된 문서 + 질문
    | prompt                                                   # 프롬프트 템플릿 적용
    | llm                                                      # 언어모델로 답변 생성
    | StrOutputParser()                                        # 결과를 문자열로 변환
)

print("✅ RAG 체인 생성 완료!")

# 체인 구조 설명
print("\n📋 생성된 RAG 체인 구조:")
print("1. 📥 입력: 사용자 질문")
print("2. 🔍 검색: retriever가 관련 문서 검색")
print("3. 📝 프롬프트: 검색된 문서와 질문을 템플릿에 삽입")
print("4. 🤖 생성: LLM이 답변 생성")
print("5. 📤 출력: 문자열 형태로 최종 답변 반환")

print("\n🎉 축하합니다! RAG 시스템이 완성되었습니다!")
print("이제 네이버 뉴스 기사에 대해 질문할 수 있습니다.")


🔗 RAG 체인 생성 중...
✅ RAG 체인 생성 완료!

📋 생성된 RAG 체인 구조:
1. 📥 입력: 사용자 질문
2. 🔍 검색: retriever가 관련 문서 검색
3. 📝 프롬프트: 검색된 문서와 질문을 템플릿에 삽입
4. 🤖 생성: LLM이 답변 생성
5. 📤 출력: 문자열 형태로 최종 답변 반환

🎉 축하합니다! RAG 시스템이 완성되었습니다!
이제 네이버 뉴스 기사에 대해 질문할 수 있습니다.


---

## 🚀 RAG 시스템 테스트하기!

이제 완성된 RAG 시스템을 실제로 테스트해보겠습니다. 

### 🎯 테스트 계획

1. **기본 질문들**: 뉴스 내용에 대한 일반적인 질문
2. **상세 질문들**: 특정 정보를 찾는 질문  
3. **범위 밖 질문**: 뉴스에 없는 내용에 대한 질문 (환각 테스트)

### 💡 주의사항

- 첫 번째 실행 시 조금 시간이 걸릴 수 있습니다
- API 요금이 발생할 수 있으니 적당히 테스트하세요
- 결과가 만족스럽지 않으면 프롬프트나 검색 설정을 조정해보세요


In [11]:
# 🎯 기본 질문 테스트
print("🎯 기본 질문들로 RAG 시스템을 테스트해보겠습니다!\n")

basic_questions = [
    "이 기사의 주요 내용을 요약해주세요.",
    "기사에서 언급된 주요 인물이나 기관은 누구인가요?",
    "이 기사가 다루는 주요 이슈는 무엇인가요?"
]

for i, question in enumerate(basic_questions, 1):
    print(f"📋 테스트 {i}/3")
    print(f"❓ 질문: {question}")
    print("🔄 답변 생성 중...")
    
    try:
        # RAG 체인 실행
        response = rag_chain.invoke(question)
        
        print(f"💬 답변:")
        print("-" * 50)
        print(response)
        print("-" * 50)
        print("✅ 답변 생성 완료!\n")
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        print("💡 API 키와 인터넷 연결을 확인해주세요.\n")

print("🎉 기본 질문 테스트가 완료되었습니다!")


🎯 기본 질문들로 RAG 시스템을 테스트해보겠습니다!

📋 테스트 1/3
❓ 질문: 이 기사의 주요 내용을 요약해주세요.
🔄 답변 생성 중...
💬 답변:
--------------------------------------------------
이 기사는 출산장려책이 점점 확산되고 있는 현상을 다루고 있습니다. 정부는 저출생 대책으로 부모 급여를 인상하고, 지자체들도 아기에게 현금을 지원하는 정책을 내놓고 있습니다. 특히, 한 기업은 출산한 직원에게 1억원을 지원하는 파격적인 정책을 발표했습니다. 이 외에도 육아휴직을 늘리거나 남성 직원의 육아휴직을 의무화하는 등의 다양한 출산장려책이 시행되고 있으며, 이러한 변화가 사회적 분위기를 바꿀 것이라는 기대가 커지고 있습니다. 그러나 중소기업의 지원이 필요하다는 목소리도 함께 제기되고 있습니다.
--------------------------------------------------
✅ 답변 생성 완료!

📋 테스트 2/3
❓ 질문: 기사에서 언급된 주요 인물이나 기관은 누구인가요?
🔄 답변 생성 중...
💬 답변:
--------------------------------------------------
주어진 문맥에서 언급된 주요 인물이나 기관은 다음과 같습니다:

1. 부영그룹 - 출산한 직원에게 1억원을 지원하는 저출생 정책을 발표한 회사.
2. 이중근 - 부영그룹 회장.
3. 정부 - 저출생 대책으로 부모 급여를 인상하고 지원 정책을 시행하는 기관.
4. 인천시 - 새로 태어난 아기에게 1억원을 지원하겠다고 발표한 지자체.
5. 광주시 - 17살 될 때까지 7400만원을 지원하겠다고 발표한 지자체.

이 외에도 출산장려책을 시행하는 여러 기업과 관련된 직원들이 언급되었습니다.
--------------------------------------------------
✅ 답변 생성 완료!

📋 테스트 3/3
❓ 질문: 이 기사가 다루는 주요 이슈는 무엇인가요?
🔄 답변 생성 중...
💬 답변:
------

In [12]:
# 🔍 상세 질문 테스트
print("🔍 더 구체적인 질문들로 테스트해보겠습니다!\n")

detailed_questions = [
    "기사에서 언급된 구체적인 숫자나 통계가 있나요?",
    "정부나 공공기관의 정책이 언급되었나요?",
    "이 기사는 언제 작성되었고, 어떤 배경에서 나온 내용인가요?"
]

for i, question in enumerate(detailed_questions, 1):
    print(f"📋 상세 테스트 {i}/3")
    print(f"❓ 질문: {question}")
    print("🔄 답변 생성 중...")
    
    try:
        response = rag_chain.invoke(question)
        
        print(f"💬 답변:")
        print("-" * 50)
        print(response)
        print("-" * 50)
        print("✅ 답변 생성 완료!\n")
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        print("💡 잠시 후 다시 시도해보세요.\n")

print("🎉 상세 질문 테스트가 완료되었습니다!")


🔍 더 구체적인 질문들로 테스트해보겠습니다!

📋 상세 테스트 1/3
❓ 질문: 기사에서 언급된 구체적인 숫자나 통계가 있나요?
🔄 답변 생성 중...
💬 답변:
--------------------------------------------------
기사에서 언급된 구체적인 숫자나 통계는 다음과 같습니다:

1. 정부의 저출생 대책으로 0세 아이에게 매달 100만원의 부모 급여를 지급하며, 아이 돌까지 1년 동안 총 1520만원을 받을 수 있습니다.
2. 인천시는 새로 태어난 아기에게 18살이 될 때까지 1억원을 지원하고, 광주시는 17살이 될 때까지 7400만원을 지원하겠다고 밝혔습니다.
3. 한 그룹사는 2021년 이후 태어난 직원 자녀에게 1억원씩 총 70억원을 지원할 계획이며, 연년생이나 쌍둥이 자녀가 있을 경우 총 2억원을 받을 수 있습니다. 

이 외에도 셋째를 낳는 경우 국민주택을 제공하겠다는 계획도 언급되었습니다.
--------------------------------------------------
✅ 답변 생성 완료!

📋 상세 테스트 2/3
❓ 질문: 정부나 공공기관의 정책이 언급되었나요?
🔄 답변 생성 중...
💬 답변:
--------------------------------------------------
네, 주어진 문맥에서 정부나 공공기관의 정책이 언급되었습니다. 정부는 저출생 대책으로 매달 주는 부모 급여를 0세 아이에게 100만원으로 올렸고, 첫만남이용권과 아동수당을 포함하면 아이 돌까지 1년 동안 1520만원을 받을 수 있다고 합니다. 또한, 지자체들도 지원에 나서고 있으며, 인천시는 새로 태어난 아기에게 18살이 될 때까지 1억원을 주겠다고 발표했습니다.
--------------------------------------------------
✅ 답변 생성 완료!

📋 상세 테스트 3/3
❓ 질문: 이 기사는 언제 작성되었고, 어떤 배경에서 나온 내용인가요?
🔄 답변 생성 중...
💬 답변:
------------

In [13]:
# 🚫 환각(Hallucination) 테스트 - 기사에 없는 내용 질문
print("🚫 환각 테스트: 기사에 없는 내용에 대해 질문해보겠습니다!\n")
print("💡 좋은 RAG 시스템은 모르는 것에 대해 솔직하게 '모른다'고 답해야 합니다.")

hallucination_questions = [
    "이 기사에서 날씨에 대한 언급이 있나요?",
    "기사에서 축구나 스포츠에 대한 내용이 언급되었나요?",
    "작년 동일한 시기의 비교 데이터가 있나요?"
]

for i, question in enumerate(hallucination_questions, 1):
    print(f"📋 환각 테스트 {i}/3")
    print(f"❓ 질문: {question}")
    print("🔄 답변 생성 중...")
    
    try:
        response = rag_chain.invoke(question)
        
        print(f"💬 답변:")
        print("-" * 50)
        print(response)
        print("-" * 50)
        
        # 응답 분석
        if "주어진 정보에서" in response or "찾을 수 없습니다" in response or "없습니다" in response:
            print("✅ 올바른 응답: 모르는 것에 대해 솔직하게 답했습니다!")
        else:
            print("⚠️ 주의: 확인이 필요한 응답입니다.")
        
        print()
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}\n")

print("🎉 환각 테스트가 완료되었습니다!")
print("💡 RAG 시스템이 얼마나 신뢰할 수 있는지 확인해보셨나요?")


🚫 환각 테스트: 기사에 없는 내용에 대해 질문해보겠습니다!

💡 좋은 RAG 시스템은 모르는 것에 대해 솔직하게 '모른다'고 답해야 합니다.
📋 환각 테스트 1/3
❓ 질문: 이 기사에서 날씨에 대한 언급이 있나요?
🔄 답변 생성 중...
💬 답변:
--------------------------------------------------
주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.
--------------------------------------------------
✅ 올바른 응답: 모르는 것에 대해 솔직하게 답했습니다!

📋 환각 테스트 2/3
❓ 질문: 기사에서 축구나 스포츠에 대한 내용이 언급되었나요?
🔄 답변 생성 중...
💬 답변:
--------------------------------------------------
주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.
--------------------------------------------------
✅ 올바른 응답: 모르는 것에 대해 솔직하게 답했습니다!

📋 환각 테스트 3/3
❓ 질문: 작년 동일한 시기의 비교 데이터가 있나요?
🔄 답변 생성 중...
💬 답변:
--------------------------------------------------
주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.
--------------------------------------------------
✅ 올바른 응답: 모르는 것에 대해 솔직하게 답했습니다!

🎉 환각 테스트가 완료되었습니다!
💡 RAG 시스템이 얼마나 신뢰할 수 있는지 확인해보셨나요?
